In [1]:
import os
from collections import defaultdict
import pandas as pd

In [8]:
em = 'bb208'
time_period = 'Future'

In [2]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [10]:
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']

In [4]:
for gauge_num in range(0, 1):
    if gauge_num not in [444, 827, 888]:
        # Directory containing the files
        directory_path = home_dir2 + f'ProcessedData/IndependentEvents/NIMROD_30mins/NIMROD_2.2km_filtered_100/{gauge_num}/WholeYear/EventSet/'
        
        # Dictionary to group files with similar names (excluding the 'part' identifier)
        file_groups = defaultdict(list)

        # Loop through all the files in the directory
        for filename in os.listdir(directory_path):
            if filename.endswith('.csv'):
                # Remove the part of the filename that specifies 'partX'
                base_name = filename.rsplit('_part', 1)[0]

                # Add the file to its corresponding group in the dictionary
                file_groups[base_name].append(filename)

        # Extract and check pairs where we have both part0 and part1
        for base_name, files in file_groups.items():

            if len(files) == 2 and any('part0' in f for f in files) and any('part1' in f for f in files):
                # Sort files so that 'part0' comes first, 'part1' comes second
                sorted_files = sorted(files, key=lambda x: 'part0' not in x) # 'part0' comes first
                # print(sorted_files)

                # Print sorted files to confirm the order
                print(f"Found pair: {sorted_files[0]}, {sorted_files[1]}")

                part0 = pd.read_csv(directory_path + sorted_files[0])
                part1 = pd.read_csv(directory_path + sorted_files[1])

                col_part0 = part0['Unnamed: 0']
                col_part1 = part1['Unnamed: 0']

                if 'part0' in sorted_files[1]:
                    print("'part0' is in the second file")
                    break

                # Find the overlapping values between the two columns
                overlapping_values = set(col_part0).intersection(set(col_part1))
                if len(overlapping_values) == 0:
                    print(f"No values are overlapping {gauge_num}")

                else:
                    # Check if part1 contains only overlapping values
                    if set(col_part1).issubset(overlapping_values):
                        print(f"-- All values are overlapping. Deleting {sorted_files[1]}")
                        os.remove(directory_path + sorted_files[1])
                    else:
                        print(f"-- Non-overlapping values exist; after filtering:")
                        # Filter part1 to exclude overlapping rows
                        part1_filtered = part1[~part1['Unnamed: 0'].isin(overlapping_values)]
                        # print(f"Filtered part1 (non-overlapping rows):\n{part1_filtered}")

                        # Print the first and last index of part0 and part1_filtered for verification
                        print(f"---- part0: First index = {part0['Unnamed: 0'].iloc[0]}, Last index = {part0['Unnamed: 0'].iloc[-1]}")
                        print(f"---- part1_filtered: First index = {part1_filtered['Unnamed: 0'].iloc[0]}, Last index = {part1_filtered['Unnamed: 0'].iloc[-1]}")

                        ## Check if filtered still has any non zero values
                        count_over_005 = part1_filtered[part1_filtered['precipitation (mm)'] > 0.05].shape[0]
                        if count_over_005 >0:
                            print(f"----- Saving {gauge_num}")
                            part1_filtered.to_csv(directory_path + sorted_files[1], index=False)
                        else:
                            print("----- All 0 values, deleting sorted_files[1]")
                            os.remove(directory_path + sorted_files[1])


FileNotFoundError: [Errno 2] No such file or directory: '/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/0/WholeYear/EventSet/'